## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d

%matplotlib inline

## Dataset handling

In [2]:
df = pd.read_csv("FODS-A2.csv")
df.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,Appliances
0,21.20,33.290,19.823333,31.790000,23.463333,38.230000,20.50,31.73,19.200000,39.363333,...,29.230000,9.85,756.183333,41.833333,4.833333,40.0,-2.67,42.017180,42.017180,290
1,21.79,38.500,19.500000,40.633333,22.500000,37.900000,21.00,37.90,20.033333,47.290000,...,40.326667,6.90,754.000000,75.000000,4.000000,40.0,2.80,24.624380,24.624380,50
2,22.39,41.390,20.200000,43.790000,24.500000,39.333333,20.10,38.26,19.390000,48.090000,...,42.060000,10.10,756.433333,68.000000,5.833333,40.0,4.45,3.731260,3.731260,260
3,24.00,30.260,24.390000,26.963333,23.390000,33.400000,22.79,31.20,21.033333,40.626667,...,35.500000,19.10,760.000000,31.000000,4.000000,40.0,1.50,1.058826,1.058826,50
4,20.05,38.245,17.600000,41.000000,21.100000,37.200000,19.89,36.40,18.200000,43.560000,...,38.863333,0.10,754.600000,99.000000,1.000000,32.0,-0.10,39.248108,39.248108,30


In [3]:
df = df.to_numpy().T

In [4]:
df.shape[0]

27

In [5]:
pearsonCoeff = []
for i in range(df.shape[0] - 1):
    pearsonCoeff.append(np.corrcoef(df[i], df[26])[0][1])

In [6]:
pearsonCoeff = np.array(pearsonCoeff)

In [7]:
pearsonCoeff

array([ 4.76741514e-02,  7.67958828e-02,  1.09542373e-01, -6.62231304e-02,
        8.08919475e-02,  2.67230679e-02,  3.56405119e-02,  1.35017383e-02,
        1.24284828e-02,  1.12097719e-02,  1.15799054e-01, -8.30805830e-02,
        2.00044183e-02, -5.92536803e-02,  3.25992026e-02, -9.10402900e-02,
        4.50878114e-03, -5.12922023e-02,  9.89081076e-02, -2.97049676e-02,
       -1.57888041e-01,  9.00481953e-02, -4.84843332e-03,  9.92493066e-03,
        5.82473700e-05,  5.82473700e-05])

## Shuffling and splitting data into training and testing subsets

In [ ]:
# Shuffling the dataset
shuffled_df = df.sample(frac=1, random_state=100)

# Define a size for your train set
training_data_size = int(0.8 * len(df))

# Split your dataset
training_data = shuffled_df[:training_data_size]
testing_data = shuffled_df[training_data_size:]

In [ ]:
X_train = training_data[["MLOGP", "RDCHI"]]
X_test = testing_data[["MLOGP", "RDCHI"]]
y_train = training_data["quantitative response of LC50"]
y_test = testing_data["quantitative response of LC50"]

In [ ]:
X_train = X_train.to_numpy().reshape(X_train.shape[1], X_train.shape[0])
X_test = X_test.to_numpy().reshape(X_test.shape[1], X_test.shape[0])
y_train = y_train.to_numpy().reshape(1, -1)
y_test = y_test.to_numpy().reshape(1, -1)